In [8]:
import torch
from torch import nn
import d2l_torch as d2l

import os
os.environ["OMP_NUM_THREADS"] = "1"

这里老师讲的时候，使用的仍然是FASHION-MNIST这个简单的数据集，但是我想起来之前keras示例代码里有个猫狗分类数据集，不过现在最新的版本已经换成Xinception了。。。找找，试试那个数据集，但是pytorch读数据的时候，那个dataset类需要自己定义，可能比较花时间，找时间再搞吧

# FASHION_MNIST

## 定义网络结构

Alexnet

In [9]:
net=nn.Sequential(
    nn.Conv2d(1,96,kernel_size=11,stride=4),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nn.Conv2d(96,256,kernel_size=5,padding=2),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2),
    nn.Conv2d(256,384,kernel_size=3,padding=1),nn.ReLU(),
    nn.Conv2d(384,384,kernel_size=3,padding=1),nn.ReLU(),
    nn.Conv2d(384,256,kernel_size=3,padding=1),nn.ReLU(),
    nn.MaxPool2d(kernel_size=3,stride=2),nn.Flatten(),
    nn.Linear(6400,4096),nn.ReLU(),nn.Dropout(p=0.5),
    nn.Linear(4096,4096),nn.ReLU(),nn.Dropout(p=0.5),
    nn.Linear(4096,10))

In [12]:
x=torch.randn(1,1,224,224)

for layer in net:
    x=layer(x)
    print(f"{layer.__class__.__name__}, Output shape\t:{x.shape}")

Conv2d, Output shape	:torch.Size([1, 96, 54, 54])
ReLU, Output shape	:torch.Size([1, 96, 54, 54])
MaxPool2d, Output shape	:torch.Size([1, 96, 26, 26])
Conv2d, Output shape	:torch.Size([1, 256, 26, 26])
ReLU, Output shape	:torch.Size([1, 256, 26, 26])
MaxPool2d, Output shape	:torch.Size([1, 256, 12, 12])
Conv2d, Output shape	:torch.Size([1, 384, 12, 12])
ReLU, Output shape	:torch.Size([1, 384, 12, 12])
Conv2d, Output shape	:torch.Size([1, 384, 12, 12])
ReLU, Output shape	:torch.Size([1, 384, 12, 12])
Conv2d, Output shape	:torch.Size([1, 256, 12, 12])
ReLU, Output shape	:torch.Size([1, 256, 12, 12])
MaxPool2d, Output shape	:torch.Size([1, 256, 5, 5])
Flatten, Output shape	:torch.Size([1, 6400])
Linear, Output shape	:torch.Size([1, 4096])
ReLU, Output shape	:torch.Size([1, 4096])
Dropout, Output shape	:torch.Size([1, 4096])
Linear, Output shape	:torch.Size([1, 4096])
ReLU, Output shape	:torch.Size([1, 4096])
Dropout, Output shape	:torch.Size([1, 4096])
Linear, Output shape	:torch.Size([1,

+ 这里可以注意一下，dropout层的输出，和它作用的Linear的输出维度是一样的，都是4096，所以**输出个数没变**
+ dropout的p=0.5，所以这4096输出里，有一半输出是0，另一半是($\frac{x_i}{1-p}$),也就是正常前面的Linear输出的二倍。这样就可以保证输出的期望不变
+ 另外，这里输出的维度，第一维表示的是 batch_size,第二维才表示通道数。
+ x=torch.randn(1,1,224,224) 表示x的通道数为1（可以认为就是输入网络最初的输入通道是1，和第一层卷积层输入通道对应）
+ Conv2d, Output shape	:torch.Size([1, 96, 54, 54]) 这里第二个维度，96，表示数据经过这个卷积层之后，输出的tensor的通道数是96，和第一层卷积层输出通道数对应。

## 加载数据

由于AlexNet设置输入是224维度，所以需要对输入的数据进行resize处理，其它都一样

In [13]:
batch_size=128
train_iter,test_iter=d2l.load_data_fashion_mnist(batch_size,resize=224)

## 训练

+ 和之前基本没什么区别，就直接去训练就好了。但是图像被resize，比以前大很多，所以训练起来会很慢
+ 单纯靠自己这个没有gpu的电脑，没法跑的。就直接停掉了
+ 用colab跑一下好了。。

In [ ]:
lr, num_epochs = 0.01, 10
d2l.train_ch6(net, train_iter, test_iter, num_epochs, lr, d2l.try_gpu())

转到google colab的jupyter：<https://colab.research.google.com/drive/1OMQ3NRnDnevdE6yJ1zaoSJwESKeoUGIs#scrollTo=5U8UdZ_8NbVx>

# 猫狗数据集

主要参考：[Image classification from scratch
](https://keras.io/examples/vision/image_classification_from_scratch/)

In [1]:
!curl -O https://download.microsoft.com/download/3/E/1/3E1C3F21-ECDB-4869-8368-6DEBA77B919F/kagglecatsanddogs_3367a.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  786M  100  786M    0     0  10.3M      0  0:01:15  0:01:15 --:--:-- 10.6M.1M      0  0:01:17  0:00:23  0:00:54 10.5M1  0:00:15 10.2M


In [ ]:
!unzip -q kagglecatsanddogs_3367a.zip  # 解压  

In [3]:
!ls PetImages

Cat Dog


在使用这些图像之前，要检查一下是否有坏的图像。这一点上，torch和TensorFlow一样，都有对应的函数。

参考：
+ [Questions about Dataloader and Dataset](https://discuss.pytorch.org/t/questions-about-dataloader-and-dataset/806/2)
+ [add exception handler for DataLoader when reading a damaged image file](https://github.com/pytorch/pytorch/issues/1137)

下面是TensorFlow的示例，pytorch的暂时还没找到，可能要使用PIL这个库去预先处理一遍，读完再送入pytorch中

```python
import os

num_skipped = 0
for folder_name in ("Cat", "Dog"):
    folder_path = os.path.join("PetImages", folder_name)
    for fname in os.listdir(folder_path):
        fpath = os.path.join(folder_path, fname)
        try:
            fobj = open(fpath, "rb")
            is_jfif = tf.compat.as_bytes("JFIF") in fobj.peek(10)
        finally:
            fobj.close()

        if not is_jfif:
            num_skipped += 1
            # Delete corrupted image
            os.remove(fpath)

print("Deleted %d images" % num_skipped)
```

In [6]:
from torch.utils.data import Dataset

dir(Dataset)

['__add__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

## 其他实现

参考：
+ [Image Classification between Dog and Cat using ResNet in PyTorch.](https://dockship.io/articles/5fe306bc5a4490141dab0b4a/image-classification-between-dog-and-cat-using-resnet-in-pytorch.#article-heading-5)
+ [Pytorch CNN tutorial with cats and dogs](https://www.kaggle.com/reukki/pytorch-cnn-tutorial-with-cats-and-dogs/notebook)